In [1]:
import requests
from bs4 import BeautifulSoup  
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.common.keys import Keys
import time

In [20]:
def reviews_scraper(url):
    executable_path = 'C:/ZLIU/SIT/660/geckodriver'
    page_url=url
    driver = webdriver.Firefox(executable_path=executable_path)
    driver.implicitly_wait(10) 
    driver.get(page_url)
    
    try:
        book_title = driver.find_element_by_xpath("//*[@id='ebooksProductTitle']").text

    except Exception as e:
        try:
            book_title = driver.find_element_by_xpath('//*[@id="productTitle"]').text
        except Exception as e:
            book_title = None


    all_link_css = "#reviews-medley-footer > div.a-row.a-spacing-large > a"
    all_link = WebDriverWait(driver, 20).until(expected_conditions.element_to_be_clickable((By.CSS_SELECTOR, all_link_css)))
    all_link.click()

    #2.2 scrape positive reviews

    #2.2.1 click link for positive reviews
    all_star = driver.find_element_by_css_selector("#a-autoid-6-announce")
    time.sleep(5)
    all_star.click()

    all_positive_css = "#star-count-dropdown_6"
    all_positive = driver.find_element_by_css_selector(all_positive_css)
    all_positive.click()

    positive_review_css = driver.find_element_by_css_selector("#filter-info-section > span:nth-child(1)")
    positive_review_n = positive_review_css.text
    positive_review_n = re.search(r"of\s[0-9]?(\,)?[0-9]+", positive_review_n).group()[3:]
    positive_review_n = [str(i) for i in positive_review_n if i !=',']
    positive_review_n = int(''.join(positive_review_n))
    positive_review_n = int(positive_review_n/10)

    positive_reviews = []
    error = []

    #2.2.1 loop thr positive reviews for many pages
    for j in range(3):

        reviewslist = driver.find_elements_by_css_selector("#cm_cr-review_list > div")

        for i in range(len(reviewslist)-1):
#             time.sleep(3)
            try:
                star = reviewslist[i].find_element_by_css_selector(
                    "div > div > div:nth-child(2) > a:nth-child(1) > i > span").get_attribute('textContent')[0]
            except Exception as e:
                star = None
                error.append(('star', j, i,1))
                

            try:
                title = reviewslist[i].find_element_by_css_selector("div > div > div:nth-child(2) > a.a-size-base.a-link-normal.review-title.a-color-base.review-title-content.a-text-bold > span").text
            except Exception as e:
                title = None
                error.append(('title', j, i,1))
              
            
            try:
                format_ = reviewslist[i].find_element_by_css_selector("div > div > div.a-row.a-spacing-mini.review-data.review-format-strip > a").text
            except Exception as e:
                format_ = None
                error.append(('format', j, i,1))
           
                     
            try:
                review = reviewslist[i].find_element_by_css_selector("div > div > div.a-row.a-spacing-small.review-data > span > span").text

            except Exception as e:
                review = None
                error.append(('review', j, i,1))
                
            
            try:
                helpful = reviewslist[i].find_element_by_css_selector("div > div > div:last-child > div > span > div.a-row.a-spacing-small > span").text
                
            except Exception as e:
                helpful = None
                error.append(('helpful', j, i,1))
                
            sentiment = 1
            positive_reviews.append((book_title, star, title, format_, review, helpful, sentiment))
        
        try:
            next_page = driver.find_element_by_css_selector('div.a-form-actions.a-spacing-top-extra-large > span > div > ul > li.a-last > a')
            next_page.click()
        except:
            continue       

    all_star = driver.find_element_by_css_selector("#a-autoid-6-announce")
    time.sleep(5)
    all_star.click()

    all_negative_css = "#star-count-dropdown_7"
    all_negativ = driver.find_element_by_css_selector(all_negative_css)
    time.sleep(5)
    all_negativ.click()

    negative_review_css = driver.find_element_by_css_selector("#filter-info-section > span:nth-child(1)")
    negative_review_n = negative_review_css.text
    negative_review_n = re.search(r"of\s[0-9]?(\,)?[0-9]+", negative_review_n).group()[3:]
    negative_review_n = [str(i) for i in negative_review_n if i !=',']
    negative_review_n = int(''.join(negative_review_n))
    negative_review_n = int(negative_review_n/10)

    negative_reviews = []
    

    for j in range(3):
        reviewslist = driver.find_elements_by_css_selector("#cm_cr-review_list > div")

        for i in range(len(reviewslist)-1):
#             time.sleep(3)
            try:
                star = reviewslist[i].find_element_by_css_selector("div > div > div:nth-child(2) > a:first-child > i > span").get_attribute('textContent')[0]
               
            except Exception as e:
                star = None
                error.append(('star', j, i,0))
               
            
            try:
                title = reviewslist[i].find_element_by_css_selector("div > div > div:nth-child(2) > a.a-size-base.a-link-normal.review-title.a-color-base.review-title-content.a-text-bold > span").text
                
            except Exception as e:
                title = None
                error.append(('title', j, i,0))
               
            
            try:
                format_ = reviewslist[i].find_element_by_css_selector("div > div > div.a-row.a-spacing-mini.review-data.review-format-strip > a").text
                
            except Exception as e:
                format_ = None
                error.append(('format', j, i, 0))
               
            
            try:
                review = reviewslist[i].find_element_by_css_selector("div > div > div.a-row.a-spacing-small.review-data > span > span").text
                
            except Exception as e:
                review = None
                error.append(('review', j, i, 0))
                
            
            try:
                helpful = reviewslist[i].find_element_by_css_selector("div > div > div:last-child > div > span > div.a-row.a-spacing-small > span").text
                
            except Exception as e:
                helpful = None
                error.append(('helpful', j, i, 0))
                           
            sentiment=0
            negative_reviews.append((book_title, star, title, format_, review, helpful, sentiment))
        
        try:
            next_page = driver.find_element_by_css_selector('div.a-form-actions.a-spacing-top-extra-large > span > div > ul > li.a-last > a')
            next_page.click()
        except:
            continue

    reviews = positive_reviews + negative_reviews
    
    driver.close()

    
    return reviews,error,book_title



In [105]:
urllist=[ 
'https://www.amazon.com/Fully-Raw-Diet-Exercise-Recipes/dp/0544559118/ref=sr_1_105?qid=1572476258&s=books&sr=1-105',
'https://www.amazon.com/Animal-Vegetable-Miracle-Tenth-Anniversary/dp/0062653059/ref=sr_1_106?qid=1572476258&s=books&sr=1-106',
'https://www.amazon.com/Travels-Charley-Search-America/dp/B0055WXSQ6/ref=sr_1_107?qid=1572476258&s=books&sr=1-107',
'https://www.amazon.com/100-Count-Sugar-Calories-Weeks/dp/0062249126/ref=sr_1_108?qid=1572476258&s=books&sr=1-108',
'https://www.amazon.com/Keto-Made-Easy-Dishes-Fast/dp/1628602880/ref=sr_1_109?qid=1572476258&s=books&sr=1-109',
'https://www.amazon.com/Chariots-Gods-Erich-von-D%C3%A4niken/dp/0425166805/ref=sr_1_110?qid=1572476258&s=books&sr=1-110',
'https://www.amazon.com/Salt-Fat-Acid-Heat-Mastering/dp/1476753830/ref=sr_1_111?qid=1572476258&s=books&sr=1-111',
'https://www.amazon.com/How-Brew-Everything-Right-First/dp/0937381888/ref=sr_1_112?qid=1572476258&s=books&sr=1-112',
'https://www.amazon.com/Forks-Over-Knives-Plant-Based-Health/dp/1615190457/ref=sr_1_114?qid=1572476580&s=books&sr=1-114',
'https://www.amazon.com/Breaking-Vicious-Cycle-Intestinal-Through/dp/0969276818/ref=sr_1_115?qid=1572476580&s=books&sr=1-115',
'https://www.amazon.com/Complete-Air-Fryer-Cookbook-Amazingly/dp/1623157439/ref=sr_1_116?qid=1572476580&s=books&sr=1-116',
'https://www.amazon.com/Neal-Barnards-Program-Reversing-Diabetes/dp/1635651271/ref=sr_1_117?qid=1572476580&s=books&sr=1-117',
'https://www.amazon.com/Fat-Chance-Beating-Against-Processed/dp/0142180432/ref=sr_1_118?qid=1572476580&s=books&sr=1-118',
'https://www.amazon.com/Kellyanns-Bone-Broth-Diet-Inches/dp/1635650259/ref=sr_1_119?qid=1572476580&s=books&sr=1-119',
'https://www.amazon.com/Reboot-Joe-Juice-Diet-Healthy-ebook/dp/B00I4C4548/ref=sr_1_120?qid=1572476580&s=books&sr=1-120',
'https://www.amazon.com/Big-Book-Kombucha-Flavoring-Fermented/dp/161212433X/ref=sr_1_121?qid=1572476580&s=books&sr=1-121',
'https://www.amazon.com/Good-Calories-Bad-Controversial-Science/dp/1400033462/ref=sr_1_122?qid=1572476580&s=books&sr=1-122',
'https://www.amazon.com/Air-Fry-Everything-Foolproof-Favorites/dp/0982754043/ref=sr_1_123?qid=1572476580&s=books&sr=1-123',
'https://www.amazon.com/Skinnytaste-Fast-Slow-Knockout-Quick-Fix/dp/0553459600/ref=sr_1_124?qid=1572476580&s=books&sr=1-124',
'https://www.amazon.com/Barefoot-Contessa-Foolproof-Recipes-Trust/dp/0307464873/ref=sr_1_125?qid=1572476580&s=books&sr=1-125',
'https://www.amazon.com/Science-Good-Cooking-Illustrated-Cookbooks/dp/1933615982/ref=sr_1_126?qid=1572476580&s=books&sr=1-126',
'https://www.amazon.com/Smoking-Meat-Essential-Guide-Barbecue/dp/1770500383/ref=sr_1_127?qid=1572476580&s=books&sr=1-127',
'https://www.amazon.com/Raising-Coffee-Sofia-Recipe-CBCFO80/dp/B019DKJW42/ref=sr_1_128?qid=1572476580&s=books&sr=1-128',
'https://www.amazon.com/All-New-Square-Foot-Gardening/dp/1591862027/ref=sr_1_130?qid=1572476890&s=books&sr=1-130',

]

In [108]:
if __name__ == "__main__":
    for i in urllist:
        reviews, error,book_title = reviews_scraper(i)
        reviews = pd.DataFrame(reviews, columns =["book_title", "star", "title", "format", "review", "helpful", "sentiment"])
        reviews_error = pd.DataFrame(error, columns = ["error_item", 'page_num', 'book_num','sentiment'])
        book_title_a = book_title.replace(' ','_').replace(':', '').replace(',', '')[0:25]
        reviews.to_csv("C:/ZLIU/SIT/660/project/"+book_title_a+"_reviews.csv")
        reviews_error.to_csv("C:/ZLIU/SIT/660/project/"+book_title_a+"_reviews_error.csv")

StaleElementReferenceException: Message: The element reference of <span class="a-size-base"> is stale; either the element is no longer attached to the DOM, it is not in the current frame context, or the document has been refreshed


In [64]:
if __name__ == "__main__":
    url = "https://www.amazon.com/30-Day-Ketogenic-Cleanse-Metabolism-Whole-Food/dp/1628601167/ref=sr_1_68?qid=1572475664&s=books&sr=1-68"
    reviews, error,book_title = reviews_scraper(url)
    reviews = pd.DataFrame(reviews, columns =["book_title", "star", "title", "format", "review", "helpful", "sentiment"])
    reviews_error = pd.DataFrame(error, columns = ["error_item", 'page_num', 'book_num','sentiment'])
#     book_title_a = url[23:40]
    book_title_a = "30_Day_Ketogenic_Cleanse_Metabolism"
    reviews.to_csv("C:/ZLIU/SIT/660/project/"+book_title_a+"_reviews.csv")
    reviews_error.to_csv("C:/ZLIU/SIT/660/project/"+book_title_a+"_reviews_error.csv")

In [7]:
# reviews = pd.DataFrame(reviews, columns =["book_title", "star", "title", "format", "review", "helpful", "sentiment"])
# reviews_error = pd.DataFrame(error, columns = ["error_item", 'page_num', 'book_num','sentiment'])
# reviews.to_csv("C:/ZLIU/SIT/660/project/"+url[23:43]+"_reviews.csv")
# reviews_error.to_csv("C:/ZLIU/SIT/660/project/"+url[23:43]+"reviews_error.csv")
book_title_a = urllist[23:40]
reviews.to_csv("C:/ZLIU/SIT/660/project/"+book_title_a+"_reviews.csv")
reviews_error.to_csv("C:/ZLIU/SIT/660/project/"+book_title_a+"_reviews_error.csv")
book_title_a

TypeError: can only concatenate str (not "list") to str

In [14]:
book_title_a = book_title.replace(' ','_').replace(':', '')[0:20]
reviews.to_csv("C:/ZLIU/SIT/660/project/"+book_title_a+"_reviews.csv")
reviews_error.to_csv("C:/ZLIU/SIT/660/project/"+book_title_a+"reviews_error.csv")
book_title_a

'Craveable_Keto_Your_'

In [76]:
book_title_a = "What_Should_Make_Dinner_Everyday"
reviews.to_csv("C:/ZLIU/SIT/660/project/"+book_title_a+"_reviews.csv")
reviews_error.to_csv("C:/ZLIU/SIT/660/project/"+book_title_a+"reviews_error.csv")
book_title_a

'What_Should_Make_Dinner_Everyday'